In [2]:
from collections import Counter
import random

import numpy as np
import pyarrow.dataset as ds
import duckdb
import pandas as pd
import numpy as np
import seaborn as sns
import plotly as pl
import plotly.express as px
import json


INDICADORES = {
    "AFD": "Adequação da Formação Docente",
    "ICG": "Complexidade de Gestão da Escola",
    "IED": "Esforço Docente",
    "ATU": "Média de Alunos por Turma",
    "HAD": "Média de Horas-aula diária",
    "DSU": "Percentual de Docentes com Curso Superior",
    "TDI": "Taxas de Distorção Idade-série",
            "TRE": "Taxa de Rendimento Escolar",

}

DIMENSIONS = {
    "Ano": "NU_ANO_CENSO",
    "Nome da região geográfica": "NO_REGIAO",
    "Nome da Unidade da Federação": "NO_UF",
    "Nome da Mesorregião": "NO_MESORREGIAO",
    "Nome da Microrregião": "NO_MICRORREGIAO"
}

with open("../map_indicadores.json") as f:
    MAP_INDICADORES = json.load(f)

In [64]:
year = 2019
indicador = "TRE"
file = f"../../../data/raw/{indicador}/MUNICIPIOS/{year}.xlsx"
df_municipios = pd.read_excel(file, skiprows=8, skipfooter=6, na_values=["--"])
df_municipios = df_municipios.drop(df_municipios.columns[[1, 2, 3]], axis=1)
columns = ['NU_ANO_CENSO', 'NO_LOCALIDADE_GEOGRAFICA', 'NO_CATEGORIA', 'NO_DEPENDENCIA']
# df_municipios = df_municipios.rename(
#     columns=dict(zip(df_municipios.columns[:4], columns))
# )

#df_municipios.insert(2, "TP_LOCALIDADE_GEOGRAFICA", "Município")


#id_vars = df_municipios.columns[:5]
#value_vars = df_municipios.columns[5:]
#df_municipios = df_municipios.melt(id_vars=id_vars, value_vars=value_vars, var_name="TP_GRUPO", value_name="METRICA")

In [88]:
year = 2019
indicador = "TRE"
file = f"../../../data/raw/{indicador}/MUNICIPIOS/{year}.xlsx"
df_municipios = pd.read_excel(file, skiprows=8, skipfooter=6, na_values=["--"])

In [90]:
18*3

54

In [87]:
year = 2019
indicador = "TRE"
file = f"../../../data/raw/{indicador}/BRASIL_REGIOES_UFS/{year}.xlsx"
df_brasil = pd.read_excel(file, skiprows=8, skipfooter=6, na_values=["--"])
df_brasil

,ano,Unnamed: 1,TIPOLOCA,DEPENDAD,tap_FUN,tap_F14,tap_F04,tap_F58,tap_F00,tap_F01,...,tab_F05,tab_F06,tab_F07,tab_F08,tab_MED,tab_M01,tab_M02,tab_M03,tab_M04,tab_MNS
0,2019,Brasil,Total,Total,92.8,95.1,89.9,98.5,97.5,91.4,...,1.9,1.8,1.8,1.9,4.8,6.1,4.6,3.0,3.9,4.6
1,2019,Brasil,Urbana,Total,93.4,95.9,90.4,98.6,97.7,92.4,...,1.6,1.5,1.5,1.6,4.7,5.9,4.5,3.0,3.9,4.0
2,2019,Brasil,Rural,Total,89.1,91.0,86.1,97.8,96.4,85.9,...,4.0,3.9,4.1,3.7,6.7,8.3,6.5,4.3,4.5,7.8
3,2019,Brasil,Total,Federal,95.2,96.9,94.5,97.6,96.8,96.3,...,0.4,0.2,0.1,0.3,1.7,1.8,1.5,1.3,2.1,2.5
4,2019,Brasil,Urbana,Federal,95.2,96.9,94.5,97.6,96.7,96.3,...,0.4,0.1,0.2,0.3,1.7,1.8,1.6,1.3,2.0,2.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577,2019,Distrito Federal,Total,Estadual,92.2,94.8,89.1,98.7,99.2,85.6,...,1.6,2.4,1.7,2.4,4.9,6.2,4.8,3.2,NaN,NaN
578,2019,Distrito Federal,Urbana,Estadual,92.2,94.8,89.1,98.8,99.2,85.7,...,1.7,2.4,1.8,2.4,5.0,6.2,4.9,3.2,NaN,NaN
579,2019,Distrito Federal,Rural,Estadual,92.8,94.4,90.4,98.4,99.4,84.0,...,0.7,1.7,1.1,1.6,3.2,4.1,3.5,1.2,NaN,NaN
580,2019,Distrito Federal,Total,Privada,98.8,99.7,97.7,99.9,99.7,99.6,...,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,NaN,0.0


In [78]:
df_municipios.columns = map(str, range(df_municipios.columns.size))

18

In [79]:
df_municipios.columns[4:63-45+4]

Index(['4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16',
       '17', '18', '19', '20', '21'],
      dtype='object')

In [62]:
df_brasil = pd.read_excel(file, skiprows=8, skipfooter=6, na_values=["--"])

In [83]:
set(df_brasil["TIPOLOCA"].unique())

{'Rural', 'Total', 'Urbana'}

In [57]:
def get_tipo_localidade_geografica(localidade_geografica: str) -> str:
    if localidade_geografica.lower() == "brasil":
        return "País"
    elif localidade_geografica.lower() in {"norte", "centro-oeste", "nordeste", "sul", "sudeste"}:
        return "Região Geográfica"
    else:
        return "Unidade Federativa"

In [49]:
a

In [ ]:
df_brasil.

In [52]:
df_brasil.iloc[:,5:]

,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,...,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Ensino Médio,NaN,NaN,NaN,NaN,NaN
6,Anos Iniciais,Anos Finais,1º Ano,2º Ano,3º Ano,4º Ano,5º Ano,6º Ano,7º Ano,8º Ano,...,6º Ano,7º Ano,8º Ano,9º Ano,Total,1ª série,2ª série,3ª série,4ª série,Não-Seriado
7,tap_F14,tap_F04,tap_F58,tap_F00,tap_F01,tap_F02,tap_F03,tap_F05,tap_F06,tap_F07,...,tab_F05,tab_F06,tab_F07,tab_F08,tab_MED,tab_M01,tab_M02,tab_M03,tab_M04,tab_MNS
8,95.1,89.9,98.5,97.5,91.4,94.1,94.7,87.9,89.1,91,...,1.9,1.8,1.8,1.9,4.8,6.1,4.6,3,3.9,4.6
9,95.9,90.4,98.6,97.7,92.4,95.3,95.7,88.8,89.6,91.4,...,1.6,1.5,1.5,1.6,4.7,5.9,4.5,3,3.9,4


In [76]:
df_brasil

,NU_ANO_CENSO,NO_CODIGO,TIPOLOCA,Dependad,HAD_INF,HAD_CRE,HAD_PRE,HAD_FUN,HAD_F14,HAD_F58,...,HAD_F05,HAD_F06,HAD_F07,HAD_F08,HAD_MED,HAD_M01,HAD_M02,HAD_M03,HAD_M04,HAD_MNS
0,2016,Brasil,Total,Total,6.0,7.7,4.9,4.6,4.5,4.7,...,4.7,4.7,4.7,4.7,4.9,5.0,4.9,4.8,4.4,4.6
1,2016,Brasil,Urbana,Total,6.1,7.8,4.9,4.6,4.5,4.7,...,4.7,4.7,4.7,4.7,4.9,5.0,4.9,4.8,4.3,4.5
2,2016,Brasil,Rural,Total,4.7,5.9,4.3,4.4,4.4,4.5,...,4.5,4.5,4.5,4.5,4.9,4.9,4.9,4.8,6.9,4.9
3,2016,Brasil,Total,Federal,7.0,7.7,6.2,5.0,4.9,5.1,...,5.1,5.1,5.1,5.1,6.7,7.0,6.9,6.8,5.1,5.3
4,2016,Brasil,Urbana,Federal,7.1,7.9,6.3,5.1,4.9,5.1,...,5.1,5.1,5.1,5.1,6.4,6.8,6.6,6.4,5.1,5.2
5,2016,Brasil,Rural,Federal,4.0,4.0,4.0,4.4,4.5,4.4,...,4.0,4.5,NaN,4.5,8.4,8.7,8.5,8.4,5.5,6.6
6,2016,Brasil,Total,Estadual,5.2,7.7,5.0,4.7,4.6,4.8,...,4.8,4.8,4.8,4.8,4.8,4.9,4.8,4.7,4.1,4.0
7,2016,Brasil,Urbana,Estadual,5.3,8.4,5.1,4.8,4.6,4.8,...,4.8,4.8,4.8,4.8,4.8,4.9,4.8,4.7,4.1,3.9
8,2016,Brasil,Rural,Estadual,4.4,4.7,4.4,4.5,4.4,4.5,...,4.5,4.5,4.5,4.5,4.5,4.6,4.5,4.4,6.0,4.3
9,2016,Brasil,Total,Municipal,5.9,7.9,4.8,4.5,4.5,4.5,...,4.5,4.5,4.5,4.5,4.7,4.8,4.7,4.7,4.3,4.0


In [59]:
df_brasil = df_brasil.rename(
    columns=dict(zip(df_brasil.columns[:4], columns))
)

df_brasil.insert(
    2,
    "TP_LOCALIDADE_GEOGRAFICA",
    df_brasil["NO_LOCALIDADE_GEOGRAFICA"].apply(set_tipo_localidade_geografica)
)
id_vars = df_brasil.columns[:5]
value_vars = df_brasil.columns[5:]
df_brasil = df_brasil.melt(id_vars=id_vars, value_vars=value_vars, var_name="TP_GRUPO", value_name="METRICA")

In [62]:
df = pd.concat([df_municipios, df_brasil], axis=0)

In [63]:
df.TP_LOCALIDADE_GEOGRAFICA.unique()

array(['Município', 'País', 'Região Geográfica', 'Unidade Federativa'],
      dtype=object)